# Створення AI-агентів із постійною пам'яттю за допомогою Cognee

Цей ноутбук демонструє, як створювати інтелектуальних AI-агентів із розвиненими можливостями пам'яті, використовуючи [**cognee**](https://www.cognee.ai/) - відкриту платформу AI-пам'яті, яка поєднує графи знань, семантичний пошук і управління сесіями для створення контекстно-орієнтованих AI-систем.

## 🎯 Навчальні цілі

До кінця цього уроку ви навчитеся:
- **Створювати графи знань на основі ембеддінгів**: Перетворювати неструктурований текст у структуровані, доступні для запитів знання
- **Реалізовувати пам'ять сесій**: Створювати багатокрокові розмови з автоматичним збереженням контексту
- **Зберігати розмови**: За потреби зберігати важливі взаємодії в довгостроковій пам'яті для майбутнього використання
- **Запитувати за допомогою природної мови**: Доступ до історичного контексту та його використання в нових розмовах
- **Візуалізувати пам'ять**: Досліджувати взаємозв'язки у графі знань вашого агента


## 🏗️ Що Ви Побудуєте

У цьому навчальному посібнику ми створимо **Помічника для Кодування** з постійною пам'яттю, який:

### 1. **Створення бази знань**
   - Збирає інформацію про профіль розробника та його експертизу
   - Обробляє принципи програмування на Python та найкращі практики
   - Зберігає історичні розмови між розробниками та AI-помічниками

### 2. **Розмови з урахуванням сесії**
   - Підтримує контекст у межах кількох запитань в одній сесії
   - Автоматично кешує кожну пару запитання/відповідь для ефективного пошуку
   - Надає узгоджені, контекстуальні відповіді на основі історії розмов

### 3. **Довготривала пам'ять**
   - Зберігає важливі розмови у довготривалу пам'ять
   - Витягує релевантні спогади з бази знань та минулих сесій для інформування нових взаємодій
   - Створює зростаючу базу знань, яка покращується з часом

### 4. **Інтелектуальне витягування пам'яті**
   - Використовує семантичний пошук, орієнтований на графи, для знаходження релевантної інформації у всій збереженій базі знань
   - Фільтрує пошук за підгрупами даних (інформація про розробника vs. принципи)
   - Поєднує кілька джерел даних для надання всебічних відповідей


## 📋 Передумови та налаштування

### Системні вимоги

Перед початком переконайтеся, що у вас є:

1. **Середовище Python**
   - Python 3.9 або новіше
   - Віртуальне середовище (рекомендується)
   
2. **Кеш Redis** (Необхідний для управління сесіями)
   - Локальний Redis: `docker run -d -p 6379:6379 redis`
   - Або скористайтеся керованим сервісом Redis
   
3. **Доступ до API LLM**
   - Ключ API OpenAI або інших провайдерів (див. [документацію](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Конфігурація бази даних**
   - За замовчуванням конфігурація не потрібна. Cognee використовує файлові бази даних (LanceDB та Kuzu)
   - За бажанням ви можете налаштувати Azure AI Search як векторне сховище (див. [документацію](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Конфігурація середовища

Створіть файл `.env` у каталозі вашого проєкту з наступними змінними:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Розуміння архітектури пам'яті Cognee

### Як працює Cognee

Cognee забезпечує складну систему пам'яті, яка виходить за межі простого зберігання ключ-значення:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Основні компоненти:

1. **Граф знань**: Зберігає сутності, зв'язки та семантичні зв’язки
2. **Векторні вбудовування**: Дозволяють семантичний пошук по всій збереженій інформації
3. **Кеш сесії**: Підтримує контекст розмови в межах і між сесіями
4. **NodeSets**: Організовують дані в логічні категорії для цільового отримання

### Типи пам'яті в цьому підручнику:

- **Постійна пам'ять**: Довготривале зберігання в графі знань
- **Пам'ять сесії**: Тимчасовий контекст розмови в кеші Redis
- **Семантична пам'ять**: Пошук за схожістю на основі векторів по всіх даних


## 📦 Встановлення необхідних пакетів

Встановіть Cognee з підтримкою Redis для управління сесіями:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Ініціалізація середовища та завантаження бібліотек

Переконайтеся, що:
1. Redis запущений (наприклад, через Docker: `docker run -d -p 6379:6379 redis`)
2. Змінні середовища встановлені перед імпортом модулів кешування
3. Якщо потрібно, перезапустіть ядро та виконайте комірки по порядку

Наступна комірка:
1. Завантажить змінні середовища з `.env`
2. Налаштує Cognee з вашими параметрами LLM
3. Увімкне кешування для управління сесіями
4. Перевірить, чи всі компоненти правильно підключені


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Налаштування каталогів зберігання

Cognee використовує два окремі каталоги для своєї роботи:
- **Корінь даних**: Зберігає завантажені документи та оброблені дані
- **Корінь системи**: Містить базу даних графа знань і метадані системи

Для цього уроку ми створимо ізольовані каталоги наступним чином:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Скидання стану пам'яті

Перш ніж почати створювати нашу систему пам'яті, давайте переконаємося, що ми починаємо з чистого аркуша.

> 💡 **Порада**: Ви можете пропустити цей крок, якщо хочете зберегти існуючі спогади з попередніх запусків, коли будете використовувати цей ноутбук пізніше.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Частина 1: Створення бази знань

### Джерела даних для нашого помічника розробника

Ми будемо використовувати три типи даних для створення всебічної бази знань:

1. **Профіль розробника**: Особистий досвід і технічний бекграунд
2. **Найкращі практики Python**: Дзен Python із практичними рекомендаціями
3. **Історичні розмови**: Минулі сесії запитань і відповідей між розробниками та AI-помічниками

Ці різноманітні дані дозволяють нашому агенту:
- Розуміти технічний контекст користувача
- Застосовувати найкращі практики у рекомендаціях
- Навчатися на основі попередніх успішних взаємодій


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Перетворення даних у граф знань

Тепер ми перетворимо наш необроблений текст у структуровану пам'ять. Цей процес:

1. **Додає дані до NodeSets**: Організовує інформацію в логічні категорії
   - `developer_data`: Профіль розробника та розмови
   - `principles_data`: Найкращі практики та рекомендації з Python

2. **Запускає Cognify Pipeline**: Витягує сутності, зв'язки та створює векторні представлення
   - Визначає ключові концепції
   - Створює семантичні зв'язки між пов'язаною інформацією
   - Генерує векторні представлення

Це може зайняти кілька хвилин, поки LLM обробляє текст і будує структуру графа:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Візуалізація графа знань

Давайте дослідимо структуру нашого графа знань. Візуалізація показує:
- **Вузли**: Сутності, витягнуті з тексту (концепції, технології, люди)
- **Ребра**: Взаємозв'язки та зв'язки між сутностями
- **Кластери**: Пов'язані концепції, згруповані за семантичною схожістю

Відкрийте згенерований HTML-файл у вашому браузері, щоб інтерактивно дослідити граф:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Збагачення пам'яті за допомогою Memify

Функція `memify()` аналізує граф знань і генерує інтелектуальні правила щодо даних. Цей процес:
- Визначає шаблони та найкращі практики
- Створює практичні рекомендації на основі змісту
- Встановлює зв'язки між різними областями знань

Ці правила допомагають агенту приймати більш обґрунтовані рішення при відповіді на запитання. Захоплення другої візуалізації дозволяє порівняти, як граф стає щільнішим після збагачення.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Частина 2: Інтелектуальне отримання пам'яті

### Демонстрація 1: Інтеграція знань з різних документів

Тепер, коли наш граф знань побудований, давайте перевіримо, як Cognee об'єднує інформацію з кількох джерел для відповіді на складні запитання.

Перше запитання демонструє:
- **Семантичне розуміння**: Знаходження відповідних концепцій, навіть якщо вони явно не згадані
- **Перехресне посилання**: Поєднання профілю розробника з принципами Python
- **Контекстуальне мислення**: Застосування найкращих практик до конкретних реалізацій

### Демонстрація 2: Фільтрований пошук із NodeSets

Друге запитання показує, як орієнтуватися на конкретні підмножини графа знань:
- Використовує параметр `node_name` для пошуку лише в межах `principles_data`
- Надає зосереджені відповіді з певної області знань
- Корисно, коли потрібна інформація з конкретної доменної області


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Частина 3: Налаштування управління сесіями

### Увімкнення пам'яті розмов

Управління сесіями є важливим для збереження контексту під час кількох взаємодій. Тут ми:

1. **Ініціалізуємо контекст користувача**: Створюємо або отримуємо профіль користувача для відстеження сесії
2. **Налаштовуємо кеш-двигун**: Підключаємося до Redis для збереження історії розмов
3. **Увімкнемо змінні сесії**: Налаштовуємо змінні контексту, які зберігаються між запитами

> ⚠️ **Важливо**: Для цього необхідно, щоб Redis працював, а в середовищі було встановлено `CACHING=true`


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Допоміжна функція: Перегляд історії сесій

Ця утиліта дозволяє нам переглядати історію розмов, збережену в Redis. Це корисно для:
- Налагодження управління сесіями
- Перевірки, чи розмови кешуються
- Розуміння, який контекст доступний агенту


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Сесія 1: Лабораторія підтримки асинхронності — Перше питання

Розпочніть сесію `async-support-lab`, запитавши про дружні до телеметрії шаблони asyncio для масштабного веб-скрейпера. Граф уже знайомий з asyncio, aiohttp і практиками моніторингу, тому відповідь має відображати попередні розмови, адаптуючи її до нового запиту.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Перевірка пам'яті сесії 1 після першого обміну

Запуск `show_history(session_1)` одразу після початкового запитання підтверджує, що Cognee записав як запит, так і відповідь у Redis. Ви повинні побачити один запис із рекомендацією щодо паралельності.


In [ ]:
await show_history(session_1)

## Сесія 1: Продовження обговорення моделей даних

Далі ми запитуємо: "Коли слід обирати dataclasses, а коли Pydantic?" використовуючи той самий ідентифікатор сесії. Cognee має об'єднати принципи Python разом із попередніми розмовами про FastAPI, щоб надати детальну пораду — демонструючи, що контекст зберігається в межах названої сесії.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Підтвердження, що історія сесії 1 містить обидва оберти

Ще один виклик `show_history(session_1)` має показати два записи запитань і відповідей. Це відповідає кроку "відтворення пам'яті" в лабораторії Mem0 і доводить, що додаткові оберти продовжують той самий транскрипт.


In [ ]:
await show_history(session_1)

## Сесія 2: Обговорення дизайну — Нова сесія

Щоб продемонструвати ізоляцію між потоками, ми запускаємо `design-review-session` і запитуємо рекомендації щодо ведення журналу для аналізу інцидентів. Хоча база знань залишається тією ж, новий ідентифікатор сесії зберігає стенограми окремо.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Огляд Сесії 2 Історія

`show_history(session_2)` має відображати лише пару запит/відповідь для огляду дизайну. Порівняйте це з Сесією 1, щоб підкреслити, як Cognee зберігає незалежні стенограми, одночасно використовуючи спільний граф знань.


In [ ]:
await show_history(session_2)

## Резюме

Вітаємо! Ви щойно надали своєму асистенту з програмування справжній шар довготривалої пам’яті, що працює на базі Cognee.

У цьому посібнику ви взяли сирий контент для розробників (код, документацію, чати) і перетворили його на граф + векторну пам’ять, яку ваш агент може шукати, аналізувати та постійно вдосконалювати.

Що ви дізналися

1. **Від сирого тексту до пам’яті ШІ**: Як Cognee обробляє неструктуровані дані та перетворює їх на інтелектуальну, доступну для пошуку пам’ять, використовуючи комбіновану архітектуру векторів + графа знань.

2. **Збагачення графа за допомогою memify**: Як вийти за межі базового створення графа та використовувати memify для додавання похідних фактів і багатших зв’язків до вашого існуючого графа.

3. **Різні стратегії пошуку**: Як здійснювати запити до пам’яті за допомогою різних типів пошуку (питання-відповідь з урахуванням графа, завершення в стилі RAG, інсайти, сирі фрагменти, пошук коду тощо) залежно від потреб вашого агента.

4. **Візуальне дослідження**: Як перевіряти та налагоджувати те, що створив Cognee, за допомогою візуалізацій графа та інтерфейсу Cognee, щоб ви могли побачити, як структурована інформація.

5. **Пам’ять, що враховує сесії**: Як поєднувати контекст кожної сесії з постійною семантичною пам’яттю, щоб агенти могли запам’ятовувати між сеансами без витоку інформації між користувачами.


## Основні висновки
1. Пам'ять як граф знань, підкріплений вбудовуваннями

    - **Структуроване розуміння**: Cognee поєднує векторне сховище та графове сховище, щоб ваші дані були як пошуковими за змістом, так і пов'язаними через відносини. За замовчуванням Cognee використовує файлові бази даних (LanceDB для векторів, Kuzu для графової бази даних).

    - **Пошук з урахуванням відносин**: Відповіді можуть базуватися не лише на "схожому тексті", але й на тому, як пов'язані сутності.

    - **Жива пам'ять**: Шар пам'яті розвивається, зростає і залишається доступним для запитів як єдиний пов'язаний граф. 

2. Режими пошуку та міркування
    - **Гібридний пошук**: пошук поєднує векторну схожість, структуру графа та міркування LLM, від пошуку сирих фрагментів до відповіді на запитання з урахуванням графа.

    - **Підбір режиму до завдання**: Використовуйте режими у стилі завершення, коли потрібні відповіді природною мовою, і режими фрагментів/резюме/графів, коли ваш агент потребує сирого контексту або для власного міркування.

3. Персоналізовані агенти, що враховують сесії
    - **Контекст сесії + довготривала пам'ять**: Cognee зберігає короткостроковий контекст "нитки" окремо від довготривалої пам'яті на рівні користувача або організації. 

## Реальні застосування

1. **Вертикальні AI-агенти**

    Використовуйте цей шаблон для створення розумних доменних копілотів, які працюють на основі Cognee як ядра для пошуку та міркувань:

- **Копілоти для розробників**: Асистенти для огляду коду, аналізу інцидентів та архітектури, які переглядають код, API, проектну документацію та тікети як єдиний граф пам'яті.

- **Копілоти для клієнтів**: Агенти підтримки або успіху, які отримують інформацію з документації продукту, FAQ, нотаток CRM та минулих тікетів з урахуванням графа та з цитованими відповідями.

- **Внутрішні експертні копілоти**: Асистенти з політики, права або безпеки, які міркують на основі взаємопов'язаних правил, керівництв та історичних рішень, а не ізольованих PDF.

    Cognee чітко позиціонується як постійна, точна пам'ять для AI-агентів, забезпечуючи живий граф знань, який інтегрується за вашим агентом і замінює випадкові комбінації векторних сховищ та кастомного графового коду. 

2. **Об'єднання даних з різних джерел в одну пам'ять**

    Такий самий підхід допомагає створити єдиний шар пам'яті з розрізнених джерел:

- **Від ізольованих даних до одного графа**: Завантажуйте структуровані (наприклад, бази даних) та неструктуровані дані (наприклад, документи, чати) в єдиний граф, підкріплений вбудовуваннями, замість окремих індексів для кожної системи.

- **Міркування між джерелами з цитатами**: Виконуйте багатокрокові міркування над усім — "об'єднуйте" журнали, метрики та документи через граф — і все одно повертайте обґрунтовані відповіді з посиланнями на джерела.

- **Центри знань**: Для таких сфер, як банківська справа або освіта, Cognee вже використовується для об'єднання PDF, внутрішніх систем та даних додатків в один граф знань з векторами, щоб агенти могли відповідати на запитання з точним, цитованим контекстом. 

## Наступні кроки

Ви реалізували основний цикл пам'яті. Ось природні розширення, які ви можете спробувати самостійно (див. [документацію Cognee](https://docs.cognee.ai/) для деталей):

1. **Експериментуйте з часовою обізнаністю**: Увімкніть temporal cognify, щоб витягувати події та часові мітки з тексту.

2. **Впровадьте міркування на основі онтології**: Визначте OWL-онтологію для вашої сфери. Використовуйте підтримку онтологій Cognee, щоб витягнуті сутності та відносини були прив'язані до цієї схеми, покращуючи якість графа та відповіді, специфічні для домену.

3. **Додайте зворотний зв'язок**: Дозвольте Cognee коригувати ваги ребер графа на основі реального зворотного зв'язку від користувачів, щоб пошук покращувався з часом, а не залишався статичним.

4. **Налаштуйте для персоналізації та поведінки сесій**: Використовуйте ідентифікатори користувачів, орендарів та набори даних, щоб надати кожній людині або команді власний вигляд на спільний механізм пам'яті.

5. **Масштабуйте для складніших агентів**: Інтегруйте Cognee у фреймворки агентів, щоб створювати багатокомпонентні системи, які всі використовують один і той самий шар пам'яті. *Плагін Microsoft Agent Framework x Cognee скоро з'явиться.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, звертаємо вашу увагу, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ мовою оригіналу слід вважати авторитетним джерелом. Для критично важливої інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникли внаслідок використання цього перекладу.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
